## Imports

In [ ]:
# Imports
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from IPython.display import clear_output
AUTOTUNE = tf.data.AUTOTUNE


# Function from stack overflow which helped with warnings while running on a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

"""For the generator and discriminator, I chose to use the pix2pix generator and discriminator developed by 
Isola, Phillip, et al. 
(Isola, Phillip, et al. "Image-to-image translation with conditional adversarial networks." 
Proceedings of the IEEE conference on computer vision and pattern recognition. 2017.)
and converted to TensorFlow by TensorFlow (Tensorflow, https://www.tensorflow.org/tutorials/generative/pix2pix)"""
from shutil import copyfile
copyfile(src = "../input/tensorflow-examples/tensorflow_examples/models/pix2pix/pix2pix.py", dst = "../working/pix2pix.py")
import pix2pix

## Data Loading

In [ ]:
# Data loading setting the batch size to 1 and the image size to 256x256
batch_size = 1
img_height = 256
img_width = 256

In [ ]:
# Load in the datasets locally using tf.keras.preprocessing.image_dataset_from_directory() splitting the datasets into
# training(80%) and validation (20%) sets.

# Photo Datasets
photos_dir = '../input/monet-dataset/photo'
train_photo = tf.keras.preprocessing.image_dataset_from_directory(
    photos_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)
test_photo = tf.keras.preprocessing.image_dataset_from_directory(
    photos_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Monet Datasets
monet_dir = '../input/monet-dataset/monet'
train_monet = tf.keras.preprocessing.image_dataset_from_directory(
    monet_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)
test_monet = tf.keras.preprocessing.image_dataset_from_directory(
    monet_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)



In [ ]:
# Test if photos have been loaded in correctly and display them.

sample_photo_train = next(iter(train_photo))
sample_monet_train = next(iter(train_monet))
sample_photo_test = next(iter(test_photo))
sample_monet_test = next(iter(test_monet))

plt.figure(1)
plt.title('Photo _Train')
plt.imshow(sample_photo_train[0][0].numpy().astype("uint8"))
plt.axis('off')

plt.figure(2)
plt.title('Monet Train')
plt.imshow(sample_monet_train[0][0].numpy().astype("uint8"))
plt.axis('off')

plt.figure(3)
plt.title('Photo Test')
plt.imshow(sample_photo_test[0][0].numpy().astype("uint8"))
plt.axis('off')

plt.figure(4)
plt.title('Monet Test')
plt.imshow(sample_monet_test[0][0].numpy().astype("uint8"))
plt.axis('off')




## Data Preprocessing
To avoid overfitting a random jitter is used on the training set. The random jitter resizes the images to 286x286, randomly crops the image, then performs a random mirror on the image. After the random jitter the images are normalized to -1 to 1 for input into the model. 

In [ ]:
def random_crop(image):
    cropped_image = tf.image.random_crop(tf.squeeze(image[0]), size=[img_height, img_width, 3])
    return cropped_image

In [ ]:
def normalize(image):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    return image

In [ ]:
def random_jitter(image):
    # resize to 286x286
    image = tf.image.resize(image, [286, 286], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # random crop to 256x256
    image = random_crop(image)

    # random mirror
    image = tf.image.random_flip_left_right(image)

    return image

In [ ]:
def preprocess_image_train(image, label):
    image = random_jitter(image)
    image = normalize(image)
    return image

In [ ]:
def preprocess_image_test(image, label):
    image = normalize(image)
    return image

In [ ]:
# Code for testing the random jitter.

plt.figure(1)
plt.title('Monet painting')
plt.imshow(sample_monet_train[0][0].numpy().astype("uint8"))
plt.axis('off')

resized_image = tf.image.resize(sample_monet_train[0].numpy().astype("uint8"), [286, 286], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
plt.figure(2)
plt.title('Monet painting with resizing')
plt.imshow(resized_image[0])
plt.axis('off')

resized_crop = random_crop(resized_image)
plt.figure(3)
plt.title('Monet painting with resizing and random crop')
plt.imshow(resized_crop)
plt.axis('off')

resized_crop_flip = tf.image.random_flip_left_right(resized_crop)
plt.figure(4)
plt.title('Monet painting with resizing, random crop and random flip')
plt.imshow(resized_crop_flip.numpy().astype("uint8"))
plt.axis('off')



In [ ]:
# Apply the preprocessing, shuffling, and caching to the datasets.

buffer_size = 1000

train_photo = train_photo.map(preprocess_image_train, num_parallel_calls=AUTOTUNE).batch(1).cache().shuffle(buffer_size)

train_monet = train_monet.map(preprocess_image_train, num_parallel_calls=AUTOTUNE).batch(1).cache().shuffle(buffer_size)

test_photo = test_photo.map(preprocess_image_test, num_parallel_calls=AUTOTUNE).batch(1).cache().shuffle(buffer_size)

test_monet = test_monet.map(preprocess_image_test, num_parallel_calls=AUTOTUNE).batch(1).cache().shuffle(buffer_size)

In [ ]:
# Test if photos have been preprocessed correctly and display them.

sample_photo_train = next(iter(train_photo))
sample_monet_train = next(iter(train_monet))
sample_photo_test = next(iter(test_photo))
sample_monet_test = next(iter(test_monet))


plt.figure(1)
plt.title('Photo _Train')
plt.imshow(sample_photo_train[0] * 0.5 + 0.5)
plt.axis('off')

plt.figure(2)
plt.title('Monet Train')
plt.imshow(sample_monet_train[0] * 0.5 + 0.5)
plt.axis('off')

plt.figure(3)
plt.title('Photo Test')
plt.imshow(sample_photo_test[0][0] * 0.5 + 0.5)
plt.axis('off')

plt.figure(4)
plt.title('Monet Test')
plt.imshow(sample_monet_test[0][0] * 0.5 + 0.5)
plt.axis('off')

## Import the Tensorflow Example Pix2Pix CycleGAN Models

In [ ]:
# Load in the pix2pix generator and discriminators. We need two of each for cycle gan. 

output_channels = 3

generator_g = pix2pix.unet_generator(output_channels, norm_type='instancenorm')
generator_f = pix2pix.unet_generator(output_channels, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

In [ ]:
# Test the untrained generators on a sample of the training set and display the images. 

to_monet = generator_g(sample_photo_train)
to_photo = generator_f(sample_monet_train)
plt.figure(figsize=(8, 8))
contrast = 8

imgs = [sample_photo_train, to_monet, sample_monet_train, to_photo]
title = ['sample_photo', 'to_monet', 'sample_monet', 'to_photo']

for i in range(len(imgs)):
    plt.subplot(2, 2, i+1)
    plt.axis('off')
    plt.title(title[i])
    if i % 2 == 0:
        plt.imshow(imgs[i][0] * 0.5 + 0.5)
    else:
      plt.imshow(imgs[i][0] * 0.5 * contrast + 0.5)

plt.show()

In [ ]:
# Test the untrained discriminators on a sample of the training set and display the images.

plt.figure(figsize=(8, 8))

plt.subplot(121)
plt.title('Monet')
plt.imshow(discriminator_y(sample_photo_train)[0])

plt.subplot(122)
plt.title('Photo')
plt.imshow(discriminator_x(sample_monet_train)[0])

plt.show()

## Generator, Discriminator, and Cycle Consistency Loss

In [ ]:
# Using keras BinaryCrossentropy loss (Sigmoid cross entropy loss)
LAMBDA = 10
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
"""The discriminator loss is the calculated on the discriminator of real image and the 
discriminator of a generated image. First, the real loss is calculated using the sigmoid 
cross entropy between a tensor of all ones of the same shape as the real image and a 
tensor of the real image. Then the generated loss is calculated by the sigmoid cross 
entropy between a tensor of all ones of the same shape as a generated image and a tensor 
of a generated image. Lastly, the total discriminator loss is calculated by adding the 
real loss and the generated loss together. The total discriminator is scaled by a factor 
of 0.5 and returned. 
"""
def discriminator_loss(discriminator_real, discriminator_generated):
    real_loss = loss_obj(tf.ones_like(discriminator_real), discriminator_real)
    generated_loss = loss_obj(tf.zeros_like(discriminator_generated), discriminator_generated)
    total_disc_loss = real_loss + generated_loss
    return total_disc_loss * 0.5

In [ ]:
"""The generator loss is calculated on a generated image by creating a tensor of all 1’s 
of the same shape as the tensor of the translated image created by the generator, then 
calculating the sigmoid cross entropy between the tensor of all ones and the tensor of 
the translated image."""
def generator_loss(generated):
    return loss_obj(tf.ones_like(generated), generated)

In [ ]:
"""The cycle consistency loss is the mean absolute error calculated between a real image 
and the real image translated through both domains otherwise known as the cycled image. 
First, the absolute value tensor of the subtraction of the tensor of the real image minus 
the tensor of the cycled image is calculated. The absolute value tensor is then reduced 
to its mean. The mean is then multiplied by LAMDA (10) and returned."""
def calc_cycle_loss(real_image, cycled_image):
    loss = tf.reduce_mean(tf.abs(real_image - cycled_image)) 
    return LAMBDA * loss

In [ ]:
"""The identity loss is the mean absolute error calculated between a real image and the 
real image translated to its original domain through a generator. For example, generator 
F translates Monet paintings to photos, if a photo was inputted into generator F then 
generator F should output something similar to the photo image. The mean absolute error 
is the identity loss  between the real image and its translation to the same domain. 
The absolute value of the tensor of the real image minus the tensor of the same image 
translated to the same domain, then the reduced mean is calculated of the absolute value 
formulating the identity loss. In my project the identity loss is multiplied by LAMBDA 
and 0.5 before returning."""
def identity_loss(real_image, same_image):
    loss = tf.reduce_mean(tf.abs(real_image - same_image))
    return LAMBDA * 0.5 * loss

In [ ]:
# Initialize the optimizers with a learning rate of 2e-4 and the exponential decay (beta_1) to 0.5
generator_g_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)

## Training the CycleGAN

In [ ]:
"""Running this locally on my gpu I could finsh an 
epoch about every 200 seconds I could not get the 
GPU option running on Kaggle. I have switched to 
only 1 epoch for the Kaggle submission to complete"""
EPOCHS = 1

In [ ]:
# Generate images using whichever model and input is chosen. 
i = 0
def generate_images(model, test_input, epoch):
    if epoch is None:
        prediction = model(test_input)
        plt.figure(figsize=(12, 12))
        display_list = [test_input[0], prediction[0]]
        title = ['Input Image', 'Predicted Image']
        for i in range(2):
            plt.subplot(1, 2, i+1)
            plt.title(title[i])
            plt.imshow(display_list[i] * 0.5 + 0.5)
            plt.axis('off')
        plt.show()
    else:
        prediction = model(test_input)
        plt.figure(figsize=(12, 12))
        display_list = [test_input[0], prediction[0]]
        title = ['Input Image', 'Predicted Image']
        for i in range(2):
            plt.subplot(1, 2, i+1)
            plt.title(title[i])
            plt.imshow(display_list[i] * 0.5 + 0.5)
            plt.axis('off')
        #plt.savefig('output/training/epoch' + str(epoch) + '.jpeg')  # Images were locally saved
        plt.show()

In [ ]:
@tf.function
def train_step(real_x, real_y):
    with tf.GradientTape(persistent=True) as tape:
        # Generator G translates photo -> monet
        # Generator F translates monet -> photo.
    
        # Generate real_x -> fake_y -> cycled_x (CycleGAN)
        fake_y = generator_g(real_x, training=True)
        cycled_x = generator_f(fake_y, training=True)
    
        # Generate real_y -> fake_x -> cycled_y (CycleGAN)
        fake_x = generator_f(real_y, training=True)
        cycled_y = generator_g(fake_x, training=True)

        # Generate the same image with its respective generator for identitiy loss
        same_x = generator_f(real_x, training=True)
        same_y = generator_g(real_y, training=True)
        
        # Generate discriminator images based off real images
        disc_real_x = discriminator_x(real_x, training=True)
        disc_real_y = discriminator_y(real_y, training=True)
        
        # Generate discriminator images based off fake images
        disc_fake_x = discriminator_x(fake_x, training=True)
        disc_fake_y = discriminator_y(fake_y, training=True)
        
        # Calculate the loss of each generated based off the discriminator images
        gen_g_loss = generator_loss(disc_fake_y)
        gen_f_loss = generator_loss(disc_fake_x)
        
        # Calculate the total cycle loss
        total_cycle_loss = calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y)
    
        # Calculate the total generator losses. Same_x and Same_y are the real image passed
        # through its respective generator to its current domain.
        total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y)
        total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)
        
        # Calculate the loss of each discriminator based off the real discriminator images nad the 
        # fake discriminator images
        disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
        disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)
  
    # Calculate the gradients for generator and discriminator
    generator_g_gradients = tape.gradient(total_gen_g_loss, generator_g.trainable_variables)
    generator_f_gradients = tape.gradient(total_gen_f_loss, generator_f.trainable_variables)
  
    discriminator_x_gradients = tape.gradient(disc_x_loss, discriminator_x.trainable_variables)
    discriminator_y_gradients = tape.gradient(disc_y_loss, discriminator_y.trainable_variables)
  
    # Apply the gradients to the Adam optimizers
    generator_g_optimizer.apply_gradients(zip(generator_g_gradients, generator_g.trainable_variables))

    generator_f_optimizer.apply_gradients(zip(generator_f_gradients, generator_f.trainable_variables))
  
    discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients, discriminator_x.trainable_variables))
  
    discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients, discriminator_y.trainable_variables))

In [ ]:
# Run the train_step on the pairs of images for specified EPOCHS 
# Skip training as the notebook was run locally
"""for epoch in range(EPOCHS):
    start_epoch = time.time()

    for image_x, image_y in tf.data.Dataset.zip((train_photo, train_monet)):
        train_step(image_x, image_y)

    # Generate images showing the models improvement every epoch
    generate_images(generator_g, sample_photo_train, epoch)

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1, time.time()-start_epoch))"""

## Generate using test dataset

In [ ]:
# Call the generate_images function on the test photos set to compare data the model has not seen
# Photo -> Monet
# Skip output as the notebook was run locally and the output was saved locally.
"""for inp in test_photo.take(5):
    generate_images(generator_g, inp[0], None)"""

In [ ]:
# Call the generate_images function on the test monet set to compare data the model has not seen
# Monet -> Photo
# Skip output as the notebook was run locally and the output was saved locally.
"""for inp in test_monet.take(5):
    generate_images(generator_f, inp[0], None)"""

## Generate 7000 Monet Paintings From Images

In [ ]:
plt.ioff()
def generate_and_save(model, input_train, input_test):
    i = 0
    for photo in input_train:
        plt.imshow(model(photo)[0] * 0.5 + 0.5)
        plt.axis('off')
        plt.savefig('output/final_output/' + str(i) + '.jpeg', pad_inches=0, bbox_inches='tight')
        plt.close()
        i += 1
    for photo in input_test:
        plt.imshow(model(photo[0])[0] * 0.5 + 0.5)
        plt.axis('off')
        plt.savefig('output/final_output/' + str(i) + '.jpeg', pad_inches=0, bbox_inches='tight')
        plt.close()
        i += 1
        

In [ ]:
# Skip output as the notebook was run locally and the output was saved locally.
# generate_and_save(generator_g, train_photo, test_photo)

In [ ]:
import shutil
shutil.make_archive('images', 'zip', '../input/imsomethingofapaintermyselfoutput/output')